In [1]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing
import matplotlib.pyplot as plt
from sklearn import cross_validation
import sys
sys.path.append('D:\\kaggle\\xgboost-master\\wrapper')
import xgboost as xgb





In [2]:
# load training and test datasets
train = pd.read_csv('input/train_set.csv', parse_dates=[2,])
test = pd.read_csv('input/test_set.csv', parse_dates=[3,])
tubes = pd.read_csv('input/tube.csv')
bill_mtls = pd.read_csv('input/bill_of_materials.csv')
specs = pd.read_csv('input/specs.csv')
components = pd.read_csv('input/components.csv')


# create some new features
train['year'] = train.quote_date.dt.year
train['month'] = train.quote_date.dt.month
train['dayofweek'] = train.quote_date.dt.dayofweek




test['year'] = test.quote_date.dt.year
test['month'] = test.quote_date.dt.month
test['dayofweek'] = test.quote_date.dt.dayofweek


In [3]:
train = pd.merge(train,tubes,on = 'tube_assembly_id',how='inner')
test =  pd.merge(test,tubes,on = 'tube_assembly_id',how='inner')

In [4]:
train.describe()

,annual_usage,min_order_quantity,quantity,cost,year,month,dayofweek,diameter,wall,length,num_bends,bend_radius,num_boss,num_bracket,other
count,30213.000000,30213.000000,30213.000000,30213.000000,30213.000000,30213.000000,30213.000000,30213.000000,30213.000000,30213.000000,30213.000000,30213.000000,30213.000000,30213.000000,30213.000000
mean,120.369377,2.084699,38.389369,13.433317,2012.067256,7.107106,3.180419,17.223018,1.384782,97.647605,3.813061,42.231223,0.025254,0.003542,0.013603
std,1590.331872,12.742776,70.761392,28.663200,2.666285,2.656734,2.239762,18.125973,0.638610,63.230131,2.199564,200.313458,0.184975,0.069664,0.150840
min,0.000000,0.000000,1.000000,0.503553,1982.000000,1.000000,0.000000,3.180000,0.710000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,2.000000,3.878190,2012.000000,6.000000,1.000000,9.520000,0.890000,48.000000,2.000000,19.050000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,10.000000,6.521146,2013.000000,8.000000,3.000000,12.700000,1.240000,86.000000,3.000000,31.750000,0.000000,0.000000,0.000000
75%,2.000000,0.000000,40.000000,13.431781,2013.000000,9.000000,6.000000,19.050000,1.650000,133.000000,5.000000,50.800000,0.000000,0.000000,0.000000
max,150000.000000,535.000000,2500.000000,1000.000000,2017.000000,12.000000,6.000000,203.200000,7.900000,1333.000000,17.000000,9999.000000,5.000000,3.000000,8.000000


In [5]:
train = train[train.cost < 800]
train = train[train.length<600]

In [6]:
test.describe()

,id,annual_usage,min_order_quantity,quantity,year,month,dayofweek,diameter,wall,length,num_bends,bend_radius,num_boss,num_bracket,other
count,30235.000000,30235.000000,30235.000000,30235.000000,30235.000000,30235.000000,30235.000000,30235.000000,30235.000000,30235.000000,30235.000000,30235.000000,30235.000000,30235.000000,30235.000000
mean,15118.000000,252.125285,2.297503,37.959782,2012.057483,7.151712,3.190375,17.335842,1.375376,99.902491,3.863238,44.621570,0.025765,0.004134,0.015876
std,8728.237031,20216.732022,21.083151,70.365779,2.661283,2.644020,2.266584,18.392260,0.670536,62.808473,2.206852,244.583923,0.191343,0.075531,0.148684
min,1.000000,0.000000,0.000000,1.000000,1985.000000,1.000000,0.000000,3.180000,0.710000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7559.500000,0.000000,0.000000,2.000000,2012.000000,6.000000,1.000000,9.520000,0.890000,50.000000,2.000000,19.050000,0.000000,0.000000,0.000000
50%,15118.000000,0.000000,0.000000,10.000000,2013.000000,8.000000,3.000000,12.700000,1.240000,89.000000,4.000000,31.750000,0.000000,0.000000,0.000000
75%,22676.500000,2.000000,0.000000,40.000000,2013.000000,9.000000,6.000000,19.050000,1.650000,137.000000,5.000000,50.800000,0.000000,0.000000,0.000000
max,30235.000000,3498645.000000,2500.000000,2500.000000,2017.000000,12.000000,6.000000,203.200000,7.900000,600.000000,17.000000,9999.000000,5.000000,4.000000,6.000000


In [7]:
train.ix[train["tube_assembly_id"]=='TA-00152',"length"] = 19
train.ix[train["tube_assembly_id"]=='TA-00154',"length"] = 75
train.ix[train["tube_assembly_id"]=='TA-00156',"length"] = 24
train.ix[train["tube_assembly_id"]=='TA-01098',"length"] = 10
train.ix[train["tube_assembly_id"]=='TA-01631',"length"] = 48
train.ix[train["tube_assembly_id"]=='TA-03520',"length"] = 46
train.ix[train["tube_assembly_id"]=='TA-04114',"length"] = 135
train.ix[train["tube_assembly_id"]=='TA-17390',"length"] = 40
train.ix[train["tube_assembly_id"]=='TA-18227',"length"] = 74
train.ix[train["tube_assembly_id"]=='TA-18229',"length"] = 51

#test
test.ix[test["tube_assembly_id"]=='TA-00152',"length"] = 19
test.ix[test["tube_assembly_id"]=='TA-00154',"length"] = 75
test.ix[test["tube_assembly_id"]=='TA-00156',"length"] = 24
test.ix[test["tube_assembly_id"]=='TA-01098',"length"] = 10
test.ix[test["tube_assembly_id"]=='TA-01631',"length"] = 48
test.ix[test["tube_assembly_id"]=='TA-03520',"length"] = 46
test.ix[test["tube_assembly_id"]=='TA-04114',"length"] = 135
test.ix[test["tube_assembly_id"]=='TA-17390',"length"] = 40
test.ix[test["tube_assembly_id"]=='TA-18227',"length"] = 74
test.ix[test["tube_assembly_id"]=='TA-18229',"length"] = 51


In [8]:
train["diff_date"] =  train["quote_date"] - train["quote_date"].min()
#train["diff_month"] = train["diff_date"]/np.timedelta64(1, 'M')
train["diff_date"] = train["diff_date"]/np.timedelta64(1, 'D')
test["diff_date"] = test["quote_date"] - train["quote_date"].min()
#test["diff_month"] = test["diff_date"]/np.timedelta64(1, 'M')
test["diff_date"] = test["diff_date"]/np.timedelta64(1, 'D')

In [9]:
train[train["supplier"]=='S-0066'][:100]
train.describe()

,annual_usage,min_order_quantity,quantity,cost,year,month,dayofweek,diameter,wall,length,num_bends,bend_radius,num_boss,num_bracket,other,diff_date
count,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000,30207.000000
mean,120.392955,2.085013,38.395902,13.342470,2012.067468,7.107359,3.180985,17.220621,1.384416,97.569076,3.812792,42.225533,0.025259,0.003542,0.013474,10910.138379
std,1590.488933,12.744020,70.766861,27.177031,2.666172,2.656907,2.239615,18.125990,0.637760,61.958945,2.199664,200.332186,0.184993,0.069671,0.150428,978.332215
min,0.000000,0.000000,1.000000,0.503553,1982.000000,1.000000,0.000000,3.180000,0.710000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,2.000000,3.875408,2012.000000,6.000000,1.000000,9.520000,0.890000,48.000000,2.000000,19.050000,0.000000,0.000000,0.000000,10991.000000
50%,0.000000,0.000000,10.000000,6.521146,2013.000000,8.000000,3.000000,12.700000,1.240000,86.000000,3.000000,31.750000,0.000000,0.000000,0.000000,11240.000000
75%,2.000000,0.000000,40.000000,13.427608,2013.000000,9.000000,6.000000,19.050000,1.650000,133.000000,5.000000,50.800000,0.000000,0.000000,0.000000,11302.000000
max,150000.000000,535.000000,2500.000000,772.805929,2017.000000,12.000000,6.000000,203.200000,7.900000,587.000000,17.000000,9999.000000,5.000000,3.000000,8.000000,12520.000000


In [10]:
quant_vars = ['quantity_'+str(i) for i in range(1,9)]
bill_mtls["total_quantity"] = bill_mtls[quant_vars].sum(axis = 1)
"""

train["quantity"] = np.log(train['quantity'])
train["log1usage"] = np.log1p(train['annual_usage'])
train["log1radius"] = np.log1p(train['bend_radius'])
train["log1length"] = np.log1p(train['length'])

test["logquantity"] = np.log(test['quantity'])
test["log1usage"] = np.log1p(test['annual_usage'])
test["log1radius"] = np.log1p(test['bend_radius'])
test["log1length"] = np.log1p(test['length'])
"""

'\n\ntrain["quantity"] = np.log(train[\'quantity\'])\ntrain["log1usage"] = np.log1p(train[\'annual_usage\'])\ntrain["log1radius"] = np.log1p(train[\'bend_radius\'])\ntrain["log1length"] = np.log1p(train[\'length\'])\n\ntest["logquantity"] = np.log(test[\'quantity\'])\ntest["log1usage"] = np.log1p(test[\'annual_usage\'])\ntest["log1radius"] = np.log1p(test[\'bend_radius\'])\ntest["log1length"] = np.log1p(test[\'length\'])\n'

In [11]:
bill_mtls["total_comps"] =  (bill_mtls.count(axis = 1)-2)/2
bill_mtls['quantXcomp'] = bill_mtls['total_quantity']* bill_mtls['total_comps']
#bill_mtls = bill_mtls.drop(['total_comps','total_quantity'],axis=1)

In [12]:
specs["total_specs"] = specs.count(axis=1)-1
specs_vars = ['spec'+str(i) for i in range(1,11)]
specs.drop(specs_vars,axis=1,inplace =True)

In [13]:
#train["cus_bendsxrad"] = train["num_bends"] * train["bend_radius"]
#test["cus_bendsxrad"] = test["num_bends"] * test["bend_radius"]
#train["diaxwalxlen"] = train["diameter"] * train["wall"] * train["length"]
#test["diaxwalxlen"] = test["diameter"] * test["wall"] * test["length"]

comp_columns = []
for i in range(1,9):
    column_label = 'component_id_'+str(i)
    comp_columns.append(column_label)
    bill_mtls[column_label].replace(np.nan,' ', regex=True, inplace= True)
    bill_mtls[column_label].replace(np.nan,' ', regex=True, inplace= True)


In [14]:
"""specs.fillna(" ",inplace = True)
uniq_specs = []
for i in range(1,11):
    uniq_specs.extend(pd.unique(specs[[i]].values.ravel()))
uniq_specs = list(set(uniq_specs))
print uniq_specs"""

'specs.fillna(" ",inplace = True)\nuniq_specs = []\nfor i in range(1,11):\n    uniq_specs.extend(pd.unique(specs[[i]].values.ravel()))\nuniq_specs = list(set(uniq_specs))\nprint uniq_specs'

In [15]:
train = pd.merge(train,bill_mtls,on ='tube_assembly_id',how = 'inner')
#train = pd.merge(train,specs,on = 'tube_assembly_id',how='inner')
test = pd.merge(test,bill_mtls,on ='tube_assembly_id',how = 'inner')
#test = pd.merge(test,specs,on = 'tube_assembly_id',how='inner')

In [16]:
train["material_id"].fillna(" ",inplace = True)
test["material_id"].fillna(" ",inplace = True)

In [17]:
# drop useless columns and create labels
idx = test.id.values.astype(int)
tube_ids = train.tube_assembly_id.values
test = test.drop(['id', 'tube_assembly_id', 'quote_date'], axis = 1)
labels = train.cost.values

train = train.drop(['quote_date','cost', 'tube_assembly_id'], axis = 1)

In [18]:
obj_columns = train.loc[:, train.dtypes == object].columns.tolist()
obj_columns = [x for x in obj_columns if x not in comp_columns] #and x not in specs_cols]
obj_indexs = []
for col in obj_columns:
    obj_indexs.append(train.columns.get_loc(col))
comp_ind = []
for col in comp_columns:
    comp_ind.append(train.columns.get_loc(col))
comp_ids = components.component_id.values.tolist()
comp_ids.append(" ")


In [19]:
train.fillna(0, inplace = True)
test.fillna(0, inplace = True)


columns = train.columns.values

# convert data to numpy array
train = np.array(train)
test = np.array(test)
    

In [20]:

# label encode the categorical variables
for i in range(train.shape[1]):
    if i in obj_indexs:
        #print(i,list(train[1:5,i]) + list(test[1:5,i]))
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[:,i]) + list(test[:,i]))
        train[:,i] = lbl.transform(train[:,i])
        test[:,i] = lbl.transform(test[:,i])
        
for i in range(train.shape[1]):
    if i in comp_ind:
        #print(i,list(train[1:5,i]) + list(test[1:5,i]))
        lbl = preprocessing.LabelEncoder()
        lbl.fit(comp_ids)
        train[:,i] = lbl.transform(train[:,i])
        test[:,i] = lbl.transform(test[:,i])


In [21]:
  """  train = train.astype(float)
    test = test.astype(float)
    
    log2FloorX = np.floor(np.log2(train + 1))
    X_feats = np.append(train, log2FloorX, axis = 1)

    log3FloorX = np.floor(np.divide(np.log(train + 1),np.log(3)))    
    X_feats = np.append(X_feats, log3FloorX, axis = 1)

    log4FloorX = np.floor(np.divide(np.log(train+1),np.log(4)))
    X_feats = np.append(X_feats, log4FloorX, axis = 1)
    
    log5FloorX = np.floor(np.divide(np.log(train+1),np.log(5)))
    X_feats = np.append(X_feats, log5FloorX, axis = 1)
    
    log6FloorX = np.floor(np.divide(np.log(train+1),np.log(6)))
    X_feats = np.append(X_feats, log6FloorX, axis = 1)
    
    log7FloorX = np.floor(np.divide(np.log(train+1),np.log(7)))
    X_feats = np.append(X_feats, log7FloorX, axis = 1)
    
    log8FloorX = np.floor(np.divide(np.log(train+1),np.log(8)))
    X_feats = np.append(X_feats, log8FloorX, axis = 1)
    
    log9FloorX = np.floor(np.divide(np.log(train+1),np.log(9)))
    X_feats = np.append(X_feats, log9FloorX, axis = 1)
    
    log12FloorX = np.floor(np.divide(np.log(train+1),np.log(12)))
    X_feats = np.append(X_feats, log12FloorX, axis = 1)
    
    log13FloorX = np.floor(np.divide(np.log(train+1),np.log(13)))
    X_feats = np.append(X_feats, log13FloorX, axis = 1)
    
    logExpFloorX = np.floor(np.log(train+1))
    X_feats = np.append(X_feats, logExpFloorX, axis = 1)
    
    sqrtFloorX = np.floor(np.sqrt(train+1))
    X_feats = np.append(X_feats, sqrtFloorX, axis = 1)
    
    powX = np.power(train+1,2)
    X_feats = np.append(X_feats, powX, axis = 1)
    
    train = X_feats"""

'  train = train.astype(float)\n  test = test.astype(float)\n  \n  log2FloorX = np.floor(np.log2(train + 1))\n  X_feats = np.append(train, log2FloorX, axis = 1)\n\n  log3FloorX = np.floor(np.divide(np.log(train + 1),np.log(3)))    \n  X_feats = np.append(X_feats, log3FloorX, axis = 1)\n\n  log4FloorX = np.floor(np.divide(np.log(train+1),np.log(4)))\n  X_feats = np.append(X_feats, log4FloorX, axis = 1)\n  \n  log5FloorX = np.floor(np.divide(np.log(train+1),np.log(5)))\n  X_feats = np.append(X_feats, log5FloorX, axis = 1)\n  \n  log6FloorX = np.floor(np.divide(np.log(train+1),np.log(6)))\n  X_feats = np.append(X_feats, log6FloorX, axis = 1)\n  \n  log7FloorX = np.floor(np.divide(np.log(train+1),np.log(7)))\n  X_feats = np.append(X_feats, log7FloorX, axis = 1)\n  \n  log8FloorX = np.floor(np.divide(np.log(train+1),np.log(8)))\n  X_feats = np.append(X_feats, log8FloorX, axis = 1)\n  \n  log9FloorX = np.floor(np.divide(np.log(train+1),np.log(9)))\n  X_feats = np.append(X_feats, log9FloorX, 

In [22]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from sklearn.metrics import make_scorer

import math

lbl = preprocessing.LabelEncoder()
lbl.fit(tube_ids)
tube_ids = lbl.transform(tube_ids)
tube_ids_ser = pd.Series( (v for v in tube_ids) )




In [59]:

#if state == "cross_validation":
train = train.astype(float)
test = test.astype(float)

def rmsle_scorer(y, y0):
    y = np.power(y,16)
    y0 = np.power(y0,16)
    #y = np.expm1(y)
    #y0 = np.expm1(y0)
    x = np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y0), 2)))
    return x

score = make_scorer(rmsle_scorer, greater_is_better= True)

#cross validation split

class KLabelFolds():
    def __init__(self, labels, n_folds=3):
        self.labels = labels
        self.n_folds = n_folds

    def __iter__(self):
        unique_labels = self.labels.unique()
        cv = cross_validation.KFold(len(unique_labels), self.n_folds,shuffle = True)
        for train, test in cv:
            test_labels = unique_labels[test]
            test_mask = self.labels.isin(test_labels)
            train_mask = np.logical_not(test_mask)
            yield (np.where(train_mask)[0], np.where(test_mask)[0])
cv = KLabelFolds(tube_ids_ser, 10)


In [24]:
train_inarr = []
test_inarr = []
for x,y in cv:
    print x,y
    train_inarr.append(x)
    test_inarr.append(y)


[   24    25    26 ..., 30200 30201 30205] [    0     1     2 ..., 30203 30204 30206]
[    0     1     2 ..., 30203 30204 30206] [   24    25    26 ..., 30200 30201 30205]


In [25]:
ids_cv = []
for x,y in zip(train_inarr,test_inarr):
    print x,y
    ids_cv.extend(y.tolist())
print ids_cv

[   24    25    26 ..., 30200 30201 30205] [    0     1     2 ..., 30203 30204 30206]
[    0     1     2 ..., 30203 30204 30206] [   24    25    26 ..., 30200 30201 30205]
[0L, 1L, 2L, 3L, 4L, 5L, 6L, 7L, 8L, 9L, 10L, 11L, 12L, 13L, 14L, 15L, 16L, 17L, 18L, 19L, 20L, 21L, 22L, 23L, 33L, 34L, 35L, 36L, 37L, 38L, 39L, 40L, 43L, 44L, 45L, 46L, 47L, 48L, 49L, 50L, 51L, 52L, 53L, 54L, 55L, 56L, 57L, 58L, 60L, 61L, 62L, 63L, 64L, 65L, 66L, 67L, 68L, 69L, 70L, 71L, 72L, 73L, 75L, 76L, 86L, 87L, 88L, 89L, 102L, 103L, 104L, 105L, 106L, 107L, 108L, 109L, 110L, 111L, 128L, 129L, 130L, 131L, 132L, 133L, 134L, 135L, 136L, 137L, 138L, 139L, 140L, 141L, 142L, 143L, 144L, 145L, 146L, 147L, 148L, 149L, 150L, 151L, 152L, 153L, 154L, 155L, 156L, 157L, 158L, 159L, 160L, 161L, 162L, 163L, 164L, 165L, 166L, 167L, 168L, 169L, 170L, 171L, 172L, 173L, 174L, 175L, 176L, 177L, 178L, 179L, 180L, 181L, 182L, 183L, 184L, 185L, 186L, 187L, 188L, 189L, 190L, 191L, 193L, 194L, 195L, 196L, 197L, 198L, 199L, 200L, 209L,

In [31]:
label_log = np.power(labels,1.0/16)
#label_log = np.log1p(labels)


def xgboost_predictor(is_validation,params,num_rounds):
    plst = list(params.items())
    num_rounds = num_rounds    
    if is_validation:
        lis = []
        preds = []
        for train_ind,test_ind in zip(train_inarr,test_inarr):
                train_cv = train[train_ind]
                test_cv = train[test_ind]
                xgtrain = xgb.DMatrix(train_cv, label=label_log[train_ind])
                xgtest = xgb.DMatrix(test_cv)
                model = xgb.train(plst, xgtrain, num_rounds)
                preds1 = model.predict(xgtest)
                y =  rmsle_scorer(label_log[test_ind],preds1)
                print y 
                lis.append(y)
        sum_lis = sum(lis)
        print "RMSLE:", sum_lis/10.0
        print "Variance:", np.var(lis)
    else:
        xgtrain = xgb.DMatrix(train, label=label_log)
        xgtest = xgb.DMatrix(test)
        model = xgb.train(plst, xgtrain, num_rounds)
        preds = model.predict(xgtest)
        preds = np.power(preds,16)
        return preds.tolist()
params = {}
#xgb params1
params["objective"] = "reg:linear" 
params["eta"] = 0.008
params["min_child_weight"] = 5 
params["subsample"] = 0.8 
params["colsample_bytree"] = 0.6
params["scale_pos_weight"] = 1 
params["silent"] = 1 
params["max_depth"] = 8 
params["max_delta_step"]=2
#params["seed"] = 42
num_rounds = 9500

#xgboost_predictor(True,params,num_rounds)
        
        


In [32]:
label_log = np.power(labels,1.0/16)

from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

    
def sklearn_predict(model,is_validation):
    if is_validation:
        lis = []
        for train_ind,test_ind in zip(train_inarr,test_inarr):
                train_cv = train[train_ind]
                test_cv = train[test_ind]
                model.fit(train_cv,label_log[train_ind])
                pred = model.predict(test_cv)
                y =  rmsle_scorer(label_log[test_ind],pred)
                print y 
                lis.append(y)
        sum_lis = sum(lis)
        print "RMSLE:", sum_lis/10
        print "Variance:", np.var(lis)
    else:
        model.fit(train,label_log)
        preds = model.predict(test)
        preds = np.power(preds,16)
        return preds.tolist()

        



model_rfr = RandomForestRegressor(n_estimators=300,random_state=0,max_depth=36,max_features = 0.5,n_jobs=-1)
model_gbr = GradientBoostingRegressor(learning_rate=0.05, subsample=0.7, max_depth=10,max_features = 0.6, n_estimators=150)
model_gbr_hub = GradientBoostingRegressor(loss='huber',alpha=0.98,learning_rate=0.05, subsample=0.7,max_features = 0.6, max_depth=10, n_estimators=150)

#model_abr =  AdaBoostRegressor(DecisionTreeRegressor(random_state=0,max_depth=36,max_features = 0.8),n_estimators=100)
#model_abrf = AdaBoostRegressor(RandomForestRegressor(n_estimators=100,random_state=0,max_depth=36,max_features = 0.5,n_jobs=-1),n_estimators=20)
#model_br =  BaggingRegressor(DecisionTreeRegressor(random_state=0,max_depth=36,max_features = 0.5),n_estimators=150)
#model_abgbr = AdaBoostRegressor(GradientBoostingRegressor(learning_rate=0.05, subsample=0.7, max_depth=10,max_features = 0.6, n_estimators=250),n_estimators=20)

#model = model_gbr
#sklearn_predict(model,True)
"""
testpreds_gbr,trained_gbr = sklearn_predict(model_gbr,False)
trainpreds_gbr = trained_gbr.predict(train)

train = np.append(train,trainpreds_gbr,axis=1)
test = np.append(test,testpreds_gbr,axis=1)
"""


'\ntestpreds_gbr,trained_gbr = sklearn_predict(model_gbr,False)\ntrainpreds_gbr = trained_gbr.predict(train)\n\ntrain = np.append(train,trainpreds_gbr,axis=1)\ntest = np.append(test,testpreds_gbr,axis=1)\n'

In [37]:
label_log = np.power(labels,1.0/16)

from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
def twofold_stack(is_sklearn,model,params={},num_rounds = 9500):
    lis =[]
    if is_sklearn:
        for train_ind,test_ind in zip(train_inarr,test_inarr):
                train_cv = train[train_ind]
                test_cv = train[test_ind]
                model.fit(train_cv,label_log[train_ind])
                pred = model.predict(test_cv)
                pred = np.power(pred,16)
                print pred
                lis.extend(pred.tolist())
        print lis[:20]
        return lis
    else:
        plst = list(params.items())
        num_rounds = num_rounds
        for train_ind,test_ind in zip(train_inarr,test_inarr):
                train_cv = train[train_ind]
                test_cv = train[test_ind]
                xgtrain = xgb.DMatrix(train_cv, label=label_log[train_ind])
                xgtest = xgb.DMatrix(test_cv)
                model = xgb.train(plst, xgtrain, num_rounds)
                preds1 = model.predict(xgtest)
                preds1 = np.power(preds1,16)
                print preds1
                lis.extend(preds1.tolist())
        print lis[:20]
        return lis

In [ ]:
        

model_rfr = RandomForestRegressor(n_estimators=300,random_state=0,max_depth=36,max_features = 0.5,n_jobs=-1)
model_gbr = GradientBoostingRegressor(learning_rate=0.05, subsample=0.7, max_depth=10,max_features = 0.6, n_estimators=150)
model_gbr_hub = GradientBoostingRegressor(loss='huber',alpha=0.98,learning_rate=0.05, subsample=0.7,max_features = 0.6, max_depth=10, n_estimators=150)


train_preds = []
test_preds = []
print "GBR"
train_preds.append(twofold_stack(True,model_gbr))
test_preds.append(sklearn_predict(model_gbr,False))

print "GBR HUBER"
train_preds.append(twofold_stack(True,model_gbr_hub))
test_preds.append(sklearn_predict(model_gbr_hub,False))

print "RF"
train_preds.append(twofold_stack(True,model_rfr))
test_preds.append(sklearn_predict(model_rfr,False))

In [38]:
                  
params = {}

print "XGB1 with no seed"
params["objective"] = "reg:linear" 
params["eta"] = 0.008
params["min_child_weight"] = 5 
params["subsample"] = 0.8 
params["colsample_bytree"] = 0.6
params["scale_pos_weight"] = 1 
params["silent"] = 1 
params["max_depth"] = 8 
params["max_delta_step"]=2
#params["seed"] = 42
num_rounds = 9500

train_preds.append(twofold_stack(False,0,params))
test_preds.append(xgboost_predictor(False,params,num_rounds))

print "XGB2 with seed 42"
params["objective"] = "reg:linear" 
params["eta"] = 0.008
params["min_child_weight"] = 5 
params["subsample"] = 0.8 
params["colsample_bytree"] = 0.6
params["scale_pos_weight"] = 1 
params["silent"] = 1 
params["max_depth"] = 8 
params["max_delta_step"]=2
params["seed"] = 42
num_rounds = 9500

train_preds.append(twofold_stack(False,0,params))
test_preds.append(xgboost_predictor(False,params,num_rounds))

        

XGB1 with no seed
[ 22.00891876  12.40886402   6.65362692 ...,   4.63061476   9.27956295
  59.76561737]
[ 22.6557827   12.89939404   7.06521368 ...,   3.73328853   3.45039129
   6.25656223]
[22.00891876220703, 12.40886402130127, 6.653626918792725, 4.72008752822876, 3.5150904655456543, 3.1945371627807617, 3.0782313346862793, 3.0063724517822266, 22.056615829467773, 12.449165344238281, 6.685998439788818, 4.750631809234619, 3.5525004863739014, 3.226337432861328, 3.1111888885498047, 3.039451837539673, 28.034318923950195, 16.735088348388672, 9.45564079284668, 6.987621784210205]
XGB2 with seed 42
[ 21.95151329  12.39851475   6.66723537 ...,   4.96793032   9.99423027
  53.27570724]
[ 22.69400024  12.91590691   7.0736022  ...,   3.95226216   3.65485859
   5.86969566]
[21.951513290405273, 12.398514747619629, 6.667235374450684, 4.712160110473633, 3.5325968265533447, 3.2073276042938232, 3.074566602706909, 3.00895357131958, 21.982877731323242, 12.436393737792969, 6.700247287750244, 4.74281215667724

In [69]:
new_labels = label_log[ids_cv]

stack_train = pd.DataFrame({"ids":ids_cv,"GBR":train_preds[0],"GBR_HUBER":train_preds[1],"RFR":train_preds[2],"XGB1":train_preds[3],"XGB2":train_preds[4]})

stack_test = pd.DataFrame({"GBR":test_preds[0],"GBR_HUBER":test_preds[1],"RFR":test_preds[2],"XGB1":test_preds[3],"XGB2":test_preds[4]})
stack_train.sort("ids",inplace =True)
stack_train.drop("ids",1)

,GBR,GBR_HUBER,RFR,XGB1,XGB2
0,21.806040,21.750735,21.890787,22.008919,21.951513
1,12.208477,12.265670,12.349943,12.408864,12.398515
2,6.543544,6.545272,6.578040,6.653627,6.667235
3,4.686823,4.675326,4.673255,4.720088,4.712160
4,3.519800,3.532754,3.524854,3.515090,3.532597
5,3.275205,3.211996,3.220042,3.194537,3.207328
6,3.129431,3.110094,3.078798,3.078231,3.074567
7,3.056728,3.032555,3.008623,3.006372,3.008954
8,21.813037,21.750735,21.932457,22.056616,21.982878
9,12.212539,12.292418,12.378165,12.449165,12.436394


In [63]:
stack_train = np.array(stack_train)
stack_test = np.array(stack_test)
model_rfr = RandomForestRegressor(n_estimators=300,random_state=0,max_depth=6,n_jobs=-1)
model = model_rfr
lis = []
for train_ind,test_ind in cv:
                train_cv = stack_train[train_ind]
                test_cv = stack_train[test_ind]
                model.fit(train_cv,new_labels[train_ind])
                pred = model.predict(test_cv)
                y =  rmsle_scorer(new_labels[test_ind],pred)
                print y 
                lis.append(y)
sum_lis = sum(lis)
print "RMSLE:", sum_lis/10
print "Variance:", np.var(lis)

0.23421854247
0.25852090555
0.239739100066
0.25353966408
0.242645396038
0.235098927551
0.227635716333
0.24661881958
0.225297770619
0.24720894998
RMSLE: 0.241052379227
Variance: 0.000104459216852


In [ ]:
stack_train = np.array(stack_train)
stack_train = np.append(train,stack_train,1)
params = {}
#xgb params1
params["objective"] = "reg:linear" 
params["eta"] = 0.008
params["min_child_weight"] = 5 
params["subsample"] = 0.8 
params["colsample_bytree"] = 0.6
params["scale_pos_weight"] = 1 
params["silent"] = 1 
params["max_depth"] = 8 
params["max_delta_step"]=2
#params["seed"] = 42
num_rounds = 9500

plst = list(params.items())
lis = []
for train_ind,test_ind in cv:
            train_cv = stack_train[train_ind]
            test_cv = stack_train[test_ind]
            xgtrain = xgb.DMatrix(train_cv, label=label_log[train_ind])
            xgtest = xgb.DMatrix(test_cv)
            model = xgb.train(plst, xgtrain, num_rounds)
            preds1 = model.predict(xgtest)
            y =  rmsle_scorer(label_log[test_ind],preds1)
            print y 
            lis.append(y)
sum_lis = sum(lis)
print "RMSLE:", sum_lis/10.0
print "Variance:", np.var(lis)


In [71]:
stack_test = np.array(stack_test)
stack_test = np.append(test,stack_test,1)

xgtrain = xgb.DMatrix(stack_train, label=label_log)
xgtest = xgb.DMatrix(stack_test)
model = xgb.train(plst, xgtrain, num_rounds)
preds = model.predict(xgtest)
preds = np.power(preds,16)
preds = pd.DataFrame({"id": idx, "cost": preds})
preds.to_csv('benchmark_xgb_stack.csv', index=False)


In [ ]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
def optimize(trials):
    space = {
             'n_estimators' : hp.quniform('n_estimators', 100, 1000, 1),
             #'learning_rate': 0.05,
             'max_depth' : hp.quniform('max_depth', 5, 50, 1),
             #'min_child_weight' : hp.quniform('min_child_weight', 2, 6, 1),
             #'subsample' : hp.quniform('subsample', 0.4, 1, 0.1),
             #'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
             #'objective': 'reg:linear',
             #'silent' : 1,
             #'scale_pos_weight' :1,
             #'max_delta_step':2
             'max_features' : hp.quniform('max_features', 0.4, 1.0, 0.1),
             'random_state':0,
             'n_jobs':-1
             }
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=150)
    print """"""""""BBBBEEEEESSSSSSTTTTTTTTT"""""""""""""""""
    print best
    
x = 1
def score(params):
    print "Training with params : ",x
    global x 
    x = x + 1
    print params
    params['n_estimators'] = int(params['n_estimators'])
    #del params['n_estimators']
    lis = []

    for train_ind,test_ind in cv:
            X_train = train[train_ind]
            X_test = train[test_ind]
            y_train = label_log[train_ind]
            y_test = label_log[test_ind]
            #dtrain = xgb.DMatrix(X_train, label=y_train)
            #dvalid = xgb.DMatrix(X_test, label=y_test)
            # watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
            #model = xgb.train(params, dtrain, num_round)
            model = RandomForestRegressor(**params)
            model.fit(X_train,y_train)
            predictions = model.predict(X_test)#.reshape((X_test.shape[0], 9))
            y =  rmsle_scorer(y_test, predictions)
            print y,
            lis.append(y)
    sum_lis = sum(lis)
    sum_lis = sum_lis/2.0
    print ""
    print "RMSLE:", sum_lis
    score = sum_lis
    print "\tScore {0}\n\n".format(score)
    return {'loss': score, 'status': STATUS_OK}


#Trials object where the history of search will be stored
trials = Trials()

optimize(trials)

    
    

In [ ]:
import os
# Fix CPU affinity caused by Numpy.
# http://stackoverflow.com/questions/15639779/what-determines-whether-different-python-processes-are-assigned-to-the-same-or-d
os.system('taskset -p 0xffffffff %d' % os.getpid())